# Buffer Overflow Protections

## External Resources
- YouTube Video - [https://youtu.be/KXr38mSliAs](https://youtu.be/KXr38mSliAs)
- YouTube Podcast - [https://youtu.be/V12_ytSs81M](https://youtu.be/V12_ytSs81M)
- NotebookLM learning materials - [https://notebooklm.google.com/notebook/d9139425-ef33-4973-b923-ca6be121fa57](https://notebooklm.google.com/notebook/d9139425-ef33-4973-b923-ca6be121fa57)

## Overview
- modern kernel/OS and compilers provide various protections from buffer overflow attacks
- Reference: https://ocw.cs.pub.ro/courses/cns/labs/lab-06

## 1. ASLR - Address Space Layout Randomization
- Kernel specific protection
- loads the stack, data and program in random memory locations making it difficult (not impossible) to guess their addresses

- the following segments are possible to randomize on x86:
    - **stack** is easily randomizable, as all stack addresses are relative to `esp` or `ebp`
    - **Data segment** may be randomized, if e.g., the data segment is set to a random value
    - **Code** can only be randomized by comiling the program as Position Independent Code/Position Independent Executable
       - default for shared libraries, but otherwise executable code is usually placed at fixed addresses

### ASLR values
- Linux allows 3 options:
    - 0: disabled
    - 1: randomize stack, vdso, libraries (except data? not sure...)
    - 2: enabled (including data segment - default value)
    
### checking ASLR
- compile and run demos/aslr.cpp program to check ASLR
- run gdb-peda 

    ```bash
    aslr # run aslr command
    ASLR is OFF # output
    ```

In [17]:
! echo kali | sudo -S cat /proc/sys/kernel/randomize_va_space

0


In [2]:
! echo kali | sudo -S bash -c 'echo 2 | sudo -S tee /proc/sys/kernel/randomize_va_space'

2


In [5]:
! cat ../demos/aslr/aslr.cpp

// aslr.cpp - program to check address space layout randomization
// compile and run the program multiple times
// if you see same value for $esp, ASLR is disabled
// $ g++ -g -m 32 -o aslr.exe aslr.cpp
#include <stdio.h>
#include <iostream>
using namespace std;

int global_var = 100; // data segment
char global_name[20]; // bss segment

int main(void) {
    char buffer[] = "Hello World!";
    register int esp asm("esp");
    register int ebp asm("ebp");
    cout << "Registers info:\n";
    cout << "===============\n";
    printf("$esp = %p\n", esp);
    printf("$ebp = %p\n", ebp);
    cout << "\nData and BSS segement\n";
    cout << "======================\n";
    printf("global_var is at: %p\n", &global_var);
    printf("global_name is at: %p\n", global_name);
    cout << "\nStack variable:\n";
    cout << "===============\n";
    printf("buffer is at: %p\n", buffer);
    cout << "\nCode segment:\n";
    cout << "=============\n";
    printf("main is at %p\n", main);
    cout << "\nS

In [20]:
%%bash
input="../demos/aslr/aslr.cpp"
output="aslr.exe"
g++ -m32 $input -o $output

../demos/aslr/aslr.cpp: In function ‘int main()’:
../demos/aslr/aslr.cpp:19:21: warning: format ‘%p’ expects argument of type ‘void*’, but argument 2 has type ‘int’ [-Wformat=]
   19 |     printf("$esp = %p\n", esp);
      |                    ~^     ~~~
      |                     |     |
      |                     void* int
      |                    %d
../demos/aslr/aslr.cpp:20:21: warning: format ‘%p’ expects argument of type ‘void*’, but argument 2 has type ‘int’ [-Wformat=]
   20 |     printf("$ebp = %p\n", ebp);
      |                    ~^     ~~~
      |                     |     |
      |                     void* int
      |                    %d


In [21]:
! ./aslr.exe | tee aslr2-1.txt

Registers info:
$esp = 0xffffc460
$ebp = 0xffffc488

Data and BSS segement
global_var is at: 0x56559008
global_name is at: 0x56559010

Stack variable:
buffer is at: 0xffffc46f

Code segment:
main is at 0x5655623d

Shared library code
printf is at 0xf7c4bd30
cout is at 0xf7fbbc00


In [22]:
! ./aslr.exe | tee aslr2-2.txt

Registers info:
$esp = 0xffffc460
$ebp = 0xffffc488

Data and BSS segement
global_var is at: 0x56559008
global_name is at: 0x56559010

Stack variable:
buffer is at: 0xffffc46f

Code segment:
main is at 0x5655623d

Shared library code
printf is at 0xf7c4bd30
cout is at 0xf7fbbc00


In [23]:
# run the diff of aslr2-1.txt and aslr2-2.txt
! diff aslr2-1.txt aslr2-2.txt

In [9]:
# ldd invokes the standard dynamic linker with LD_TRACE_LOADED OBJECTS
# causes the dynamic linker to inspect the program's dynamic dependencies and the objects
! ldd ./aslr.exe

	linux-gate.so.1 (0xf7f83000)
	libstdc++.so.6 => /lib32/libstdc++.so.6 (0xf7c00000)
	libc.so.6 => /lib32/libc.so.6 (0xf7800000)
	libm.so.6 => /lib32/libm.so.6 (0xf7e5a000)
	/lib/ld-linux.so.2 (0xf7f85000)
	libgcc_s.so.1 => /lib32/libgcc_s.so.1 (0xf7e33000)


In [13]:
# see the library loaded in different addresses
# run it few times to see the random addresses...
! ldd ./aslr.exe

	linux-gate.so.1 (0xf7fc3000)
	libstdc++.so.6 => /lib32/libstdc++.so.6 (0xf7c00000)
	libc.so.6 => /lib32/libc.so.6 (0xf7800000)
	libm.so.6 => /lib32/libm.so.6 (0xf7e9a000)
	/lib/ld-linux.so.2 (0xf7fc5000)
	libgcc_s.so.1 => /lib32/libgcc_s.so.1 (0xf7e73000)


### Set ASLR to option 0
- disable ASLR

In [24]:
! echo kali | sudo -S bash -c 'echo 0 | sudo tee /proc/sys/kernel/randomize_va_space'

0


In [25]:
! echo kali | sudo -S cat /proc/sys/kernel/randomize_va_space

0


In [26]:
! ./aslr.exe | tee aslr0-1.txt

Registers info:
$esp = 0xffffc460
$ebp = 0xffffc488

Data and BSS segement
global_var is at: 0x56559008
global_name is at: 0x56559010

Stack variable:
buffer is at: 0xffffc46f

Code segment:
main is at 0x5655623d

Shared library code
printf is at 0xf7c4bd30
cout is at 0xf7fbbc00


In [27]:
! ./aslr.exe | tee aslr0-2.txt

Registers info:
$esp = 0xffffc460
$ebp = 0xffffc488

Data and BSS segement
global_var is at: 0x56559008
global_name is at: 0x56559010

Stack variable:
buffer is at: 0xffffc46f

Code segment:
main is at 0x5655623d

Shared library code
printf is at 0xf7c4bd30
cout is at 0xf7fbbc00


In [28]:
! diff aslr0-1.txt aslr0-2.txt
# no difference!

### ldd command

- print shared object dependencies
- https://man7.org/linux/man-pages/man1/ldd.1.html
- don't run ldd on untrusted ELF
- use objectdump instead

```bass
 $ objdump -p /path/to/program | grep NEEDED
 ```

In [29]:
! ldd ./aslr.exe

	linux-gate.so.1 (0xf7fc7000)
	libstdc++.so.6 => /lib32/libstdc++.so.6 (0xf7c00000)
	libc.so.6 => /lib32/libc.so.6 (0xf7800000)
	libm.so.6 => /lib32/libm.so.6 (0xf7e9e000)
	/lib/ld-linux.so.2 (0xf7fc9000)
	libgcc_s.so.1 => /lib32/libgcc_s.so.1 (0xf7e77000)


In [30]:
! ldd ./aslr.exe

	linux-gate.so.1 (0xf7fc7000)
	libstdc++.so.6 => /lib32/libstdc++.so.6 (0xf7c00000)
	libc.so.6 => /lib32/libc.so.6 (0xf7800000)
	libm.so.6 => /lib32/libm.so.6 (0xf7e9e000)
	/lib/ld-linux.so.2 (0xf7fc9000)
	libgcc_s.so.1 => /lib32/libgcc_s.so.1 (0xf7e77000)


In [30]:
# detailed but not as good results as ldd
! objdump -p ./aslr.exe


./aslr.exe:     file format elf32-i386

Program Header:
    PHDR off    0x00000034 vaddr 0x00000034 paddr 0x00000034 align 2**2
         filesz 0x00000180 memsz 0x00000180 flags r--
  INTERP off    0x000001b4 vaddr 0x000001b4 paddr 0x000001b4 align 2**0
         filesz 0x00000013 memsz 0x00000013 flags r--
    LOAD off    0x00000000 vaddr 0x00000000 paddr 0x00000000 align 2**12
         filesz 0x00000580 memsz 0x00000580 flags r--
    LOAD off    0x00001000 vaddr 0x00001000 paddr 0x00001000 align 2**12
         filesz 0x000005a0 memsz 0x000005a0 flags r-x
    LOAD off    0x00002000 vaddr 0x00002000 paddr 0x00002000 align 2**12
         filesz 0x00000348 memsz 0x00000348 flags r--
    LOAD off    0x00002eb4 vaddr 0x00003eb4 paddr 0x00003eb4 align 2**12
         filesz 0x00000158 memsz 0x00000174 flags rw-
 DYNAMIC off    0x00002ec0 vaddr 0x00003ec0 paddr 0x00003ec0 align 2**2
         filesz 0x00000100 memsz 0x00000100 flags rw-
    NOTE off    0x000001c8 vaddr 0x000001c8 paddr 0x00000

#### Allow ptrace processes
**ptrace** - allows parent process to observe and control the execution of another process
- used primarily to implement breakpoint debugging and system call tracing

In [32]:
# check the ptrace value
! echo kali | sudo -S sysctl kernel.yama.ptrace_scope

[sudo] password for kali: kernel.yama.ptrace_scope = 0


In [33]:
# set the ptrace value
! echo kali | sudo -S sysctl -w kernel.yama.ptrace_scope=0

[sudo] password for kali: kernel.yama.ptrace_scope = 0


### 2. PIE
- Position Independent Executable
- randomizes Code segment base address
- randomizes GOT (Global Offset Table for global data-variables) and PLT (Procedure Linkage Table)
    - PLT addresses are fixed during  linking to load the libc.so library codes/functions
    - shared libraries are position independent code (PIC); they don't know where they are being loaded
- disable PIE in gcc/g++ using **-no-pie** flag

### 3. Executable Stack Protection (NX) also called (DEP)
- also called DEP - Data Execution Prevention
- in newer g++, programs must be compiled by disabling stack protector to execute code injected into stack
- check executable stack:
`$ readelf -l <filename>`
- note the line: 
GNU_STACK      0x000000 0x00000000 0x00000000 0x00000 0x00000 RW  0x10
RW 
- RW - Read and Write but not Executable
- RWE - Read, Write, and Executable

In [33]:
%%bash
in=../demos/hello.cpp
out=hello.exe
g++ -g -m32 -o $out $in 

readelf -l $out | grep GNU_STACK

  GNU_STACK      0x000000 0x00000000 0x00000000 0x00000 0x00000 RW  0x10


#### enable RWX/disable DEP
- compile the program using -z execstack switch of gcc/g++

In [ ]:
%%bash
in=../demos/hello.cpp
out=hello1.exe
g++ -g -m32 -z execstack -o $out $in 

readelf -l $out | grep GNU_STACK

  GNU_STACK      0x000000 0x00000000 0x00000000 0x00000 0x00000 RWE 0x10


### 4. Stack Canaries
- "canary in a coal mine" to detect carbon monoxide by using a warm blooded animal like a bird
- place a small random integer just before the stack return address
    - if the value is overwritten/corrupted during the function call, there's a stack overflow attempt!
- use **-fno-stack-protector** flag in gcc/g++ to disable canaries


### checksec command

- program to check security settings on an ELF
- GDB/PEDA provides checksec command
- to run from bash, need to install checksec

In [36]:
! echo kali | sudo -S apt install checksec

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  checksec
0 upgraded, 1 newly installed, 0 to remove and 14 not upgraded.
Need to get 21.7 kB of archives.
After this operation, 104 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 checksec all 2.1.0+git20191113.bf85698-2 [21.7 kB]
Fetched 21.7 kB in 1s (42.9 kB/s)   

78Selecting previously unselected package checksec.
(Reading database ... 70774 files and directories currently installed.)
Preparing to unpack .../checksec_2.1.0+git20191113.bf85698-2_all.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [ 20%] [###########...............................................] 8Unpacking checksec (2.1.0+git20191113.bf85698-2) ...
7Progress: [ 40%] [#######################...................................] 8Setting up checksec (2.1.0+git20191

In [37]:
! checksec --file=./aslr.exe

RELRO           STACK CANARY      NX            PIE             RPATH      RUNPATH	Symbols		FORTIFY	Fortified	Fortifiable  FILE
Full RELRO      Canary found      NX enabled    PIE enabled     No RPATH   No RUNPATH   83 Symbols     Yes	0		1	./aslr.exe


### High level explanation

- https://blog.siphos.be/2011/07/high-level-explanation-on-some-binary-executable-security/

#### RELRO - Relocation Read Only

- the following resource provides good overview and example of RELRO
- https://www.redhat.com/en/blog/hardening-elf-binaries-using-relocation-read-only-relro
- Global Offset Table (GOT) and Procedure Linkage Table (PLT) can be made Read only

#### FORTIFY

- use compiler functionality to fortify certain variables that could potentially be overflown
- depends on the intelligence/feature of the compiler
